# Data Engineering Capstone Project: Find datasets

## Purpose of this notebook

I use this notebook to sketch ideas, test functions and build prototypes. The aim is to quickly implement ideas and use parts to build the actually used python scripts. Therefore, the notebook prioritizes quick implementation over standards or best practices. 



## Problem setting

> A SME wants to access their sales data on a cloud computing platform to make further analyses and build a dashboard to inform managers.



## Data 

1. Yahoo! Finance
    1. get historical and daily data on certain industries
2. Google trends
    1. get weekly trends of the last 5 years
    


# Manage API Queries

## Helper functions

In [2]:
import sys
sys.path.insert(1, '../src/data')

import helper_functions as h

#### process_query_results

In [128]:
import numpy as np 
from datetime import datetime

def handle_query_results(df_query_result, keywords, query_return_length=261):
    """Process query results: 
            (i) check for empty response --> create df with 0s if empty
            (ii) drop isPartial rows and column
            (iii) transpose dataframe to wide format (keywords//search interest)
    
    Input
        df: dataframe containing query result (could be empty)
        filename: name of temporary file
        query_return_length: 261 is normal return length of query result 
        
    Return
        Dataframe: contains query results in long format 
        (rows: keywords, columns: search interest over time)
    """
    # non-empty df
    if df_query_result.shape[0] != 0:
        # reset_index to preserve date information, drop isPartial column
        df_query_result_processed = df_query_result.reset_index()\
            .drop(['isPartial'], axis=1)

        df_query_result_long = pd.melt(df_query_result_processed, id_vars=['date'], var_name='keyword', value_name='search_interest')
        
        # long format (date, keyword, search interest)
        return df_query_result_long

    # no search result for any keyword: empty df
    else:        
        # create empty df with 0s
        df_zeros = pd.DataFrame(np.zeros((query_return_length*len(keywords), 3)), columns=['date','keyword', 'search_interest'])
        # replace 0s with keywords
        df_zeros['keyword'] = np.repeat(keywords, query_return_length)

        return df_zeros

In [105]:
df_zeros = pd.DataFrame() 
handle_query_results(df_zeros, keywords=['desirable Zoetis', 'resilient Zoetis', 'robust Zoetis', 'reasonable Zoetis', 'strong Zoetis'])

,date,keyword,search_interest
0,0.0,desirable Zoetis,0.0
1,0.0,desirable Zoetis,0.0
2,0.0,desirable Zoetis,0.0
3,0.0,desirable Zoetis,0.0
4,0.0,desirable Zoetis,0.0
...,...,...,...
1300,0.0,strong Zoetis,0.0
1301,0.0,strong Zoetis,0.0
1302,0.0,strong Zoetis,0.0
1303,0.0,strong Zoetis,0.0


## Firm names
### Retrieve

In [30]:
import pandas as pd
import re
from datetime import datetime


def get_firms_sp500():
    """Obtain S&P 500 listings from Wikipedia"""
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df_sp500 = table[0]
    
    return df_sp500


def regex_strip_legalname(raw_names):
    """Removes legal entity, technical description or firm type from firm name
    
    Input
        raw_names: list of strings with firm names
        
    Return
        list of strings: firm names without legal description 
    
    """
    
    pattern = r"(\s|\.|\,|\&)*(\.com|Enterprise|Worldwide|Int\'l|N\.V\.|LLC|Co\b|Inc\b|Corp\w*|Group\sInc|Group|Company|Holdings\sInc|\WCo(\s|\.)|plc|Ltd|Int'l\.|Holdings|\(?Class\s\w+\)?)\.?\W?"
    stripped_names = [re.sub(pattern,'', n) for n in raw_names]
    
    return stripped_names

# get firm S&P500 from Wikipedia
keep_columns = ['Symbol','Security', 'GICS Sector']
df_sp500_wiki = get_firms_sp500().loc[:,keep_columns]

# rename column, set ticker as index
df_sp500_wiki= df_sp500_wiki.rename(columns={'Symbol': 'ticker', 'Security': 'firm_name_raw', 'GICS Sector': 'sector'})

# process firm names (remove legal entity, suffix)
df_sp500_wiki['firm_name_processed'] = regex_strip_legalname(list(df_sp500_wiki.firm_name_raw))

# add retrieval date 
df_sp500_wiki['date_get_firmname'] = datetime.today().strftime('%Y-%m-%d')

# drop duplicate firm names 
df_sp500_wiki.drop_duplicates(subset='firm_name_processed', inplace=True)

h.make_csv(df_sp500_wiki, 'firm_namess.csv', '../data/processed',header=True)

Path created: ../data/processed/firm_namess.csv


## ESG topics
### Define

In [29]:
import pandas as pd
from datetime import datetime

topics_negative = ['scandal', 'greenwashing', 'corruption', 'fraud', 
                   'bribe', 'tax', 'forced', 'harassment', 'violation', 
                   'illegal', 'conflict', 'weapons', 'pollution',
                   'inequality', 'discrimination', 'sexism', 'racist', 
                   'intransparent', 'nontransparent', 'breach', 'lawsuit', 
                   'unfair', 'bad', 'problem', 'hate', 'issue', 'controversial', 
                  'strike', 'scam', 'trouble', 'controversy', 'mismanagement', 
                  'crisis', 'turmoil', 'shock', 'whistleblow', 'dispute']

topics_positive =  ['green', 'sustainable', 'positive', 'best', 'good', 
                    'social', 'charity', 'ethical', 'renewable', 'carbon neutral', 
                   'equitable', 'ecological', 'efficient', 'improve', 'cooperative', 
                   'beneficial', 'collaborative', 'productive', 'leader', 
                   'donate', 'optimal', 'favorable', 'desirable', 'resilient', 
                   'robust', 'reasonable', 'strong', 'organic']

print("Defined {} negative and {} positive topics".format(len(topics_negative), len(topics_positive)))


# create df with topics and label
df_topics_neg = pd.DataFrame({'topic':topics_negative, 'positive': 0})
df_topics_pos = pd.DataFrame({'topic':topics_positive, 'positive': 1})
df_topics = pd.concat([df_topics_neg, df_topics_pos]).reset_index(drop=True)
df_topics['date_define_topic'] = datetime.today().strftime('%Y-%m-%d')

h.make_csv(df_sp500_wiki, 'esg_topics.csv', '../data/processed',header=True)

Defined 37 negative and 28 positive topics


,topic,positive
0,scandal,0
1,greenwashing,0
2,corruption,0
3,fraud,0
4,bribe,0


## Construct keywords

In [34]:
import numpy as np

# expand firm names for each topic
df_sp500_expanded = df_sp500_wiki.iloc[np.repeat(np.arange(len(df_sp500_wiki)), len(df_topics))]
# expand topics for each firm 
df_topics_expanded = df_topics.iloc[list(np.arange(len(df_topics)))*len(df_sp500_wiki)]\
    .set_index(df_sp500_expanded.index)

# GENERATE search keywords as a combintation of firm name + topic
search_terms = pd.DataFrame({'search_term':[i+' '+j for j in df_sp500_wiki.firm_name_processed for i in df_topics.topic]})\
    .set_index(df_sp500_expanded.index)

# merge topics, firm names, and search terms into 1 df
df_query_input = pd.concat([df_topics_expanded, df_sp500_expanded, search_terms], axis=1).reset_index(drop=True)

df_query_input.head()

,topic,positive,ticker,firm_name_raw,sector,firm_name_processed,date_get_firmname,search_term
0,scandal,0,MMM,3M Company,Industrials,3M,2020-10-17,scandal 3M
1,greenwashing,0,MMM,3M Company,Industrials,3M,2020-10-17,greenwashing 3M
2,corruption,0,MMM,3M Company,Industrials,3M,2020-10-17,corruption 3M
3,fraud,0,MMM,3M Company,Industrials,3M,2020-10-17,fraud 3M
4,bribe,0,MMM,3M Company,Industrials,3M,2020-10-17,bribe 3M



## API queries with: `query()` 
### handles errors, manages timeout and stores indices where errors occured 

In [129]:
from time import sleep
from random import randint # for random timeout +/- 5

def query(keywords, filepath, max_retries=1, idx_unsuccessful=list(), timeout=20) :
    """Handle failed query and handle raised exceptions
    
    Input
        keywords: list with keywords for which to retrieve news
        max_retries: number of maximum retries
        until_page: maximum number of retrievd news page
        
    
    Return
        Inidces where max retries were reached
    """    
    # retry until max_retries reached
    for attempt in range(max_retries):   

        # random int from range around timeout 
        timeout_randomized = randint(timeout-3,timeout+3)

        try:
            df_result = query_googletrends(keywords)


        # handle query error
        except Exception as e:

            # increase timeout
            timeout += 5

            print(">>> EXCEPTION at {}: {} \n Set timeout to {}\n".format(i, e, timeout))
            # sleep
            h.sleep_countdown(timeout_randomized, print_step=2)


        # query was successful: store results, sleep 
        else:

            # generate timestamp for csv
            stamp = h.timestamp_now()

            # merge news dataframes and export query results
            h.make_csv(df_result, "gtrends.csv", filepath, append=True)

            # sleep
            h.sleep_countdown(timeout_randomized)
            break

    # max_retries reached: store index of unsuccessful query
    else:
        h.make_csv(pd.DataFrame(keywords), "unsuccessful_queries.csv", filepath, append=True)
        print("i: {} appended to idx_unsuccessful\n".format(keywords))

## Google Trends Query

### Raw data

In [89]:
from pytrends.request import TrendReq
import pandas as pd 

# initialize pytrends
pt = TrendReq()

# pass keywords to pytrends API 
kw_batch_null = ['desirable Zoetis', 'resilient Zoetis', 'robust Zoetis', 'reasonable Zoetis', 'strong Zoetis']
kw_batch_success = ['cake', 'pizza']

pt.build_payload(kw_list=kw_batch_success) 

# store results from query in df and append to df_list
df_query_result = pt.interest_over_time()

### Query API with method

In [118]:
def query_googletrends(keywords):
    # initialize pytrends
    pt = TrendReq()
    
    # pass keywords to api
    pt.build_payload(kw_list=keywords) 
    
    # store results from query in df and append to df_list
    df_query_result = pt.interest_over_time()
    
    # preprocess query results
    df_query_result_processed = handle_query_results(df_query_result, keywords)
    
    return df_query_result_processed

In [130]:
# for each batch of 5 keywords, make googletrends query 
for i in range(0,len(df_query_input)-4)[:2]:
    kw_batch = [k for k in df_query_input.search_term[i:i+5]]
    
    print(i, kw_batch)
    
    # feed keyword batch to api query function
    query(keywords=kw_batch, filepath='../data/raw')

0 ['scandal 3M', 'greenwashing 3M', 'corruption 3M', 'fraud 3M', 'bribe 3M']
Path created: ../data/raw/gtrends.csv
 Complete!emaining:
1 ['greenwashing 3M', 'corruption 3M', 'fraud 3M', 'bribe 3M', 'tax 3M']
Path created: ../data/raw/gtrends.csv
 Complete!emaining:


### Preprocess data 

In [78]:
# reset_index to preserve date information, drop isPartial column
df_query_result_processed = df_query_result.reset_index()\
    .drop(['isPartial'], axis=1)

df_query_result_long = pd.melt(df_query_result_processed, id_vars=['date'], var_name='keyword', value_name='search_interest')
df_query_result_long.head()

KeyError: "['isPartial'] not found in axis"

In [149]:
for i in range(0,16,5):
    print(df_query_input.iloc[i:i+5,:].topic)

0         scandal
1    greenwashing
2      corruption
3           fraud
4           bribe
Name: topic, dtype: object
5           tax
6        forced
7    harassment
8     violation
9       illegal
Name: topic, dtype: object
10          conflict
11           weapons
12         pollution
13        inequality
14    discrimination
Name: topic, dtype: object
15            sexism
16            racist
17     intransparent
18    nontransparent
19            breach
Name: topic, dtype: object


In [ ]:
from pytrends.request import TrendReq
import time # for sleep and timestamp
import pandas as pd

def googletrends_query(batched_keywords, sec_sleep=30):
    """Get Google trends data in a reliable way
        if server does not respond, store results so far and retry with increased timeout
    
    Input
        batched_keywords: list of keywords with chunks of five
        temp_data: name of temporary file in ./data/ directory (defined in make_csv())

    Return
        None: stores query results as .csv in ./data/ 
    """
    # initialize pytrends
    pt = TrendReq()
    
    # empty list to store dataframes
    df_list = []
    
    ## iterate over keyword batches to obtain query results
    for i, batch in enumerate(batched_keywords):
    
        # make query
        try:
            # pass keywords to pytrends API 
            pt.build_payload(kw_list=batch) 

            # store results from query in df and append to df_list
            df_query_result = pt.interest_over_time()
            
            # check if empty and transpose to long format
            df_query_result_long = handle_query_results(df_query_result, batch)
            df_list.append(df_query_result_long)
            
            # wait (timeout)
            time.sleep(sec_sleep)
        
        # error handling
        except Exception as e:
            print("Error {} for batch {}".format(e, i))
            
            # merge results fetched so far
            df_query_result = pd.concat(df_list)
            
            # store results in csv, indicating last successful batch (i-1) and timestamp
            timestr = time.strftime("%Y%m%d-%H%M%S")
            df_filename = '{}_googletrends_batch_{}.csv'.format(timestr, i-1)            
            make_csv(df_query_result, df_filename, 'data', index=True, header=True)
            print("Store results in", df_filename)
            
            # recursively call function with keyword_batches starting from i
            # and an increased timeout by 10 seconds
            sec_sleep += 10
            print("Increased sec_sleep to {}".format(sec_sleep))
            google_query(batched_keywords[i:], sec_sleep+10)
    
    
    ## finally store in csv
    # merge query results
    df_all_results = pd.concat(df_list)
    # store results in csv, indicating last batch (i) and timestamp
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df_filename = '{}_googletrends_batch_{}.csv'.format(timestr, i)  
    
    make_csv(df_all_results, df_filename, 'data', index=True)

# Yahoo! Finance

To complement the "soft" or alternaitve indicators from Google Trends, I rely on financial metrics. 

## Goal: Collect financial metrics of a firm.

I retrieve financial indicators from Yahoo! Finance and a Python wrapper of their API called `yahooquery`. It takes symbols called "ticker" as input which the stock exchange uses to identify firms. For example, the ticker for the company 3M is *MMM*. 

Following the steps below gets relevant financial data from Yahoo! Finance.

1. query all available financial data with the *ticker* from `metadata` as input
2. select most recent date of financials
3. handle missings
    1. inspect and ignore columns with missings
    2. crosscheck that no missings exists in final dataframe


In [119]:
# import helper functions from ../src/data
from sys import path
path.insert(1, '../src/data')
import helper_functions as h

from yahooquery import Ticker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
print("Ensure pandas version>= 1.1.3. Your version: {}".format(pd.__version__))

Ensure pandas version>= 1.1.3. Your version: 1.1.3


## Collect and export raw financial data

In [121]:
# metadata file in ../data/raw
path_meta = h.get_files("../data/raw", name_contains="*meta*", absolute_path=False)[0]

# read in firm tickers from metadata  (serves as input for queries)
df_meta = pd.read_csv(path_meta)

# get ticker
tickers = list(df_meta.ticker.unique())

# take sample for fast protoyping
# n_sample = 10

print("Tickers as input for queries:")
print(tickers) #SAMPLING [:n_sample]

query = Ticker(tickers)#[:n_sample]

# query all financial data
df_fin_raw = query.all_financial_data()

# store raw data
stamp = h.timestamp_now()
h.make_csv(df_fin_raw, stamp+'yahoofinance.csv', '../data/raw', header=True, index=True)

Ensure pandas version>= 1.1.3. Your version: 1.1.3
Tickers as input for queries:
['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY']
Path created: ../data/raw/20201022-160824yahoofinance.csv


## Preprocessing

In [5]:
load_file = h.get_files('../data/raw', name_contains="*yahoo*" , absolute_path=False)[0]
print(f"Load file {load_file}")

# select most recent date
df_fin_raw = pd.read_csv(load_file, index_col=0)
print("Loaded file has {} unique firm ticker".format(df_fin_raw.index.nunique()))

print("Select most recent date for each firm")
date_most_recent = df_fin_raw.groupby(df_fin_raw.index).asOfDate.max() 
df_fin_recent = df_fin_raw[df_fin_raw.asOfDate.isin(date_most_recent)]

Load file ../data/raw\20201022-160824yahoofinance.csv
Loaded file has 59 unique firm ticker
Select most recent date for each firm


### Handle missings

In [163]:
# inspect missings
df_colmiss = h.inspect_missings(df_fin_recent, verbose=True)

MISSINGS
----------------------------------------
Any missing in any row: 59/59 (100.0 %)

Return info on column missings
                                  missing_count       share
CurrentDeferredTaxesLiabilities              59  100.000000
OtherEquityInterest                          59  100.000000
LiabilitiesHeldforSaleNonCurrent             59  100.000000
OtherInventories                             59  100.000000
NetIncomeExtraordinary                       59  100.000000
...                                         ...         ...
CommonStockEquity                             1    1.694915
ShareIssued                                   1    1.694915
StockholdersEquity                            1    1.694915
TangibleBookValue                             1    1.694915
DilutedNIAvailtoComStockholders               1    1.694915

[237 rows x 2 columns]
****************************************


In [194]:
# count missings per column, store in series
count_column_missing = df_fin_recent.isna().sum(axis=0).sort_values(ascending=False)
column_missing_names = count_column_missing.index

# get array of missing count with unique values to loop over
count_column_missing_unique = count_column_missing[count_column_missing >= 0].sort_values().unique()

# define until which missing count should be iterated
max_missing = 20
plot_x, plot_y = np.zeros(max_missing), np.zeros(max_missing)

print("Trade off rows against columns\nDrop rows and modifiy original dataframe to keep more columns\n")
print("Keep Columns\tDrop Rows [%]")
for arr_idx, i in enumerate(count_column_missing_unique[:max_missing]):
    select_columns = count_column_missing[count_column_missing <= i].index

    # return indices (=ticker) where the 1 missing per column occurs
    indices_many_missings = list(df_fin_recent.loc[df_fin_recent[select_columns].isnull().any(1),:].index)
    
    ## compare modified vs. raw dropped missings
    # modified df 
    df_fin_mod = df_fin_recent.drop(indices_many_missings).dropna(axis=1)
    df_fin_nomiss = df_fin_mod
    # raw df
    df_raw = df_fin_recent.dropna(axis=1)
    
    ### Benefits of dropping x rows
    ## comparison of dropping and keeping
    # original 
    n_row_orig, n_col_orig = df_fin_recent.shape
    # modified & cleaned 
    n_row_mod, n_col_mod = df_fin_mod.shape
    # raw & cleaned
    n_row_nomod, n_col_nomod = df_fin_raw_nomiss.shape
    
    ## STATISTICS
    # dropped rows to modify df (count and %)
    n_dropped_rows_mod = len(indices_many_missings)
    pct_dropped_rows_mod = round(n_dropped_rows_mod/n_row_orig*100)
    
    # dropped cols for modified df (count and %)
    n_dropped_cols_mod = n_col_orig - n_col_mod
    pct_dropped_cols_mod = round(n_dropped_cols_mod/n_col_orig*100)
    
    # dropped cols for non-modified df (count and %)
    n_dropped_cols_nomod = n_col_orig - n_col_nomod
    pct_dropped_cols_nomod = round(n_dropped_cols_nomod/n_col_orig*100)
    
    # plot dropped rows against retained columns 
    # (x=100-pct_dropped_rows_mod, y=pct_dropped_rows_mod)
    plot_x[arr_idx], plot_y[arr_idx] = 100-pct_dropped_cols_mod, pct_dropped_rows_mod
    
    
    print("{}\t({})\t{}".format(plot_x[arr_idx],i, plot_y[arr_idx])) 


# plot
# plt.plot(plot_y, plot_x)
# plt.ylabel("% retained columns")
# plt.xlabel("% dropped rows")
# plt.show(block=True)

Trade off rows against columns
Drop rows and modifiy original dataframe to keep more columns

Keep Columns	Drop Rows [%]
2.0	(0)	0.0
14.0	(1)	5.0
20.0	(2)	10.0
26.0	(3)	24.0
27.0	(4)	25.0
29.0	(5)	36.0
30.0	(6)	42.0
37.0	(7)	53.0
37.0	(8)	56.0
39.0	(9)	61.0
41.0	(10)	68.0
44.0	(11)	75.0
45.0	(12)	81.0
46.0	(13)	86.0
49.0	(14)	90.0
49.0	(15)	90.0
51.0	(16)	92.0
51.0	(18)	93.0
51.0	(19)	93.0
59.0	(20)	97.0


In [208]:
def inspect_drop_rows_retain_columns(data, max_missing=3):
    """Dropping rows with many missings for certain columns 
        to keep columns
    :param data: dataframe
    :param max_missing: defines until which column-wise missing count should be iterated
    :return list with indices to drop, tuple of numpy arrays for plotting: Columns to keep vs. rows dropped (%)
    """
    # count missings per column, store in series
    count_column_missing = data.isna().sum(axis=0).sort_values(ascending=False)
    column_missing_names = count_column_missing.index

    # get array of missing count with unique values to loop over
    count_column_missing_unique = count_column_missing[count_column_missing >= 0].sort_values().unique()

    # define until which column-wise missing count should be iterated
    plot_x, plot_y = np.zeros(max_missing), np.zeros(max_missing)
    print("Trade off rows against columns\nDrop rows and modifiy original dataframe to keep more columns\n")
    print("i\tKeep Columns\tDrop Rows [%]\n"+'-'*40)
    
    drop_rows = []
    for arr_idx, i in enumerate(count_column_missing_unique[:max_missing]):
        select_columns = count_column_missing[count_column_missing <= i].index

        # return indices (=ticker) where the 1 missing per column occurs
        indices_many_missings = list(data.loc[data[select_columns].isnull().any(1),:].index)
        drop_rows.append(indices_many_missings)
        
        ## compare modified vs. raw dropped missings
        # modified df 
        df_fin_mod = data.drop(indices_many_missings).dropna(axis=1)
        df_fin_nomiss = df_fin_mod
        # raw df
        df_raw = data.dropna(axis=1)

        ### Benefits of dropping x rows
        ## comparison of dropping and keeping
        # original 
        n_row_orig, n_col_orig = data.shape
        # modified & cleaned 
        n_row_mod, n_col_mod = df_fin_mod.shape
        # raw & cleaned
        n_row_nomod, n_col_nomod = df_fin_raw_nomiss.shape

        ## STATISTICS
        # dropped rows to modify df (count and %)
        n_dropped_rows_mod = len(indices_many_missings)
        pct_dropped_rows_mod = round(n_dropped_rows_mod/n_row_orig*100)

        # dropped cols for modified df (count and %)
        n_dropped_cols_mod = n_col_orig - n_col_mod
        pct_dropped_cols_mod = round(n_dropped_cols_mod/n_col_orig*100)

        # dropped cols for non-modified df (count and %)
        n_dropped_cols_nomod = n_col_orig - n_col_nomod
        pct_dropped_cols_nomod = round(n_dropped_cols_nomod/n_col_orig*100)

        # plot dropped rows against retained columns 
        # (x=100-pct_dropped_rows_mod, y=pct_dropped_rows_mod)
        plot_x[arr_idx], plot_y[arr_idx] = 100-pct_dropped_cols_mod, pct_dropped_rows_mod
        print("{}\t{}\t\t{}".format(i, plot_x[arr_idx], plot_y[arr_idx])) 
        

    return drop_rows, plot_x, plot_y

In [212]:
rows_many_missings, _,_ = inspect_drop_rows_retain_columns(df_fin_recent, max_missing=3)
print("Drop rows:", rows_many_missings[1])

# select rows that cause 1 missing for certain columns
drop_rows = rows_many_missings[1]

# drop firms with missings
df_fin_recent_clean = df_fin_recent.drop(drop_rows)

print(f"Exclude {len(drop_rows)} // {len(df_fin_recent_clean)} firms still available\n")

# inspect missings again
df_colmiss = h.inspect_missings(df_fin_recent_clean)

# drop columns with missings
df_fin_nomiss = df_fin_recent_clean.drop(df_colmiss.index, axis=1)

# crosscheck
h.inspect_missings(df_fin_nomiss)
print("Awesome!")

print("Financial data 1 row for 1 firm with shape:")
print(df_fin_nomiss.shape)

Trade off rows against columns
Drop rows and modifiy original dataframe to keep more columns

i	Keep Columns	Drop Rows [%]
----------------------------------------
0	2.0		0.0
1	14.0		5.0
2	20.0		10.0
Drop rows: ['AEE', 'APA', 'AXP']
Exclude 3 // 56 firms still available

MISSINGS
----------------------------------------
Any missing in any row: 56/56 (100.0 %)

Return info on column missings
                                      missing_count       share
CurrentDeferredTaxesLiabilities                  56  100.000000
OtherInventories                                 56  100.000000
CurrentNotesPayable                              56  100.000000
NetIncomeExtraordinary                           56  100.000000
OtherEquityInterest                              56  100.000000
...                                             ...         ...
EndCashPosition                                   1    1.785714
InvestingCashFlow                                 1    1.785714
TotalDebt                     

In [198]:
def row_missing_count(df, top_n=None):
    """Inspect absolute and relative missings across rows
    Args
        df: pandas DataFrame
        top_n: restrict output to top_n indices with most missings across columns 
    Return
        pandas dataframe with indices and their absolute and relative missings across columns
    
    """
    
    df_colmiss_idx = df.T.isna().sum().sort_values(ascending=False)[:top_n]
    df_colmiss_idx_share = df_colmiss_idx/df.shape[1]
    
    return pd.concat([df_colmiss_idx, df_colmiss_idx_share], axis=1, keys=['missing_count', 'missing_share'])

row_missing_count(df_fin_recent, top_n=10)

,missing_count,missing_share
symbol,,
AEE,226,0.937759
AZO,179,0.742739
AFL,150,0.622407
AIG,141,0.585062
AMP,140,0.580913
AIZ,133,0.551867
AXP,131,0.543568
ANTM,131,0.543568
ALL,127,0.526971


### Reshape to long format and export

In [125]:
# reshape into long format
df_fin_long = pd.melt(df_fin_nomiss, id_vars=['asOfDate', 'periodType'], var_name='financial_var', value_name='financial_val', ignore_index=False)
df_fin_clean = df_fin_long.rename(columns={'asOfDate': 'date_financial', 'periodType': 'financial_interval'})
h.make_csv(df_fin_clean, 'yahoofinance_long.csv', '../data/processed', header=True, index=True)

Path created: ../data/processed/yahoofinance_long.csv


# Temp dump

In [37]:
def inspect_missings(data, verbose=True):
    """Inspect missings across rows and across columns
    
    Args 
        data: pandas dataframe 
        
    Returns
        :return : dataframe with info on column missings  
    """
    if verbose:
        print("MISSINGS")
        print('-'*40)
        
    # check rows
    rows_all = data.shape[0]
    rows_nomiss = data.dropna().shape[0]

    rowmiss_count = rows_all - rows_nomiss
    rowmiss_share = rowmiss_count/rows_all*100

    if verbose:
        print("Any missing in any row: {}/{} ({} %)".format(rowmiss_count,rows_all, rowmiss_share))
        print()
    
    # check columns
    col_miss = [col for col in data.columns if data[col].isna().any()]
    # no missings for any column
    if not col_miss:
        print("No missings for any column.")
    else:
        # print share of missings for each column
        print("Return info on column missings")
        ds_colmiss = data.loc[:,col_miss].isna().sum()
        ds_colmiss_relative = data.loc[:,col_miss].isna().sum()/rows_all*100
        
        df_colmiss = pd.concat([ds_colmiss, ds_colmiss_relative], axis=1, keys=['missing_count', 'share'])\
                        .sort_values("share", ascending=False)
        if verbose:
            print(df_colmiss)
            print('*'*40)
            
        return df_colmiss